In [1]:
!nvidia-smi

Sat Jul 15 21:32:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.1 MB/s eta 0:00:00


In [3]:
import os
import csv
import requests
from tqdm import tqdm
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration

In [4]:
from google.colab import drive
drive.mount('/content/drive')

! unzip -q '/content/drive/MyDrive/cleaned_img.zip' -d '/content/data' # Load Preprocessed Images of Memes

Mounted at /content/drive


In [5]:
# Load Pre-trained Image Captioning Model BLIP-2
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b-coco")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b-coco").to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
def generate_caption(image):
  inputs = processor(image, return_tensors="pt").to("cuda")
  out = model.generate(**inputs)
  generated_text = processor.decode(out[0], skip_special_tokens=True)

  return generated_text

def write_csv(data):
    with open(caption_csv, 'a+') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(data)

In [7]:
img_dir = "/content/data"
caption_csv = "/content/BLIP_2_caption.csv"

# Extract captions from memes and save them in csv
for i,img_path in enumerate(tqdm(os.listdir(img_dir))):
        if img_path.endswith('.png'):
            img = os.path.join(img_dir, img_path)
            raw_image = Image.open(img)
            caption = generate_caption(raw_image)
            data = ['img/'+img_path, caption[:-1]]
            write_csv(data)

  0%|          | 0/12140 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 12140/12140 [1:49:04<00:00,  1.86it/s]


In [10]:
# Save the csv file in Google Drive
!cp -r /content/BLIP_2_caption.csv /content/drive/MyDrive